# Объединение текстовых и визуальных данных

Этот блокнот выполняет слияние двух датасетов:
1. Текстовых данных (результат preprocessing)
2. Эмбеддингов изображений

## Процесс объединения

1. **Загрузка данных**:
   - `train_text` - текстовый датасет из файла `df_train.parquet`
   - `train_image` - датасет эмбеддингов изображений из `image_dataset.parquet`

2. **Предобработка изображений**:
   - Фильтрация только валидных записей (`is_valid == 1`)
   - Удаление дубликатов по полю `id`

3. **Двухэтапное слияние**:
   - Первое объединение по полю `base_title_image`
   - Второе объединение по полю `cand_title_image`
   - Для каждого изображения сохраняются соответствующие эмбеддинги в отдельных колонках

4. **Финализация**:
   - Удаление служебных колонок `is_valid`
   - Сохранение результата в `all_data.parquet`

In [1]:
import pandas as pd

In [ ]:
# Подгружаеем части данных

train_text = pd.read_parquet('df_train.parquet')
train_image = pd.read_parquet('image_dataset.parquet')
train_image = train_image[train_image['is_valid'] == 1]
train_image = train_image.drop_duplicates(subset='id')
merge_df = train_text.merge(train_image.rename(columns={'id': 'base_title_image', 'embeddings': 'base_img_embeddings'}),
                 on='base_title_image', how='left')

merge_df = merge_df.merge(train_image.rename(columns={'id': 'cand_title_image', 'embeddings': 'cand_img_embeddings'}),
                 on='cand_title_image', how='left')

merge_df = merge_df.drop(['is_valid_x', 'is_valid_y'], axis=1)

merge_df.to_parquet('all_data.parquet', index=False)

In [3]:
print(merge_df)

                                              base_item_id  \
0        13ade32c3e614d939faad4ab68350bc52ce8848b7a64bd...   
1        13ade32c3e614d939faad4ab68350bc52ce8848b7a64bd...   
2        13ade32c3e614d939faad4ab68350bc52ce8848b7a64bd...   
3        d0b78018657dff01508954bb58d4f03f1ddf11525d8d26...   
4        d0b78018657dff01508954bb58d4f03f1ddf11525d8d26...   
...                                                    ...   
1879550  b52354ea9479e14d53cd017521d3c2aaa84f085e67572a...   
1879551  b52354ea9479e14d53cd017521d3c2aaa84f085e67572a...   
1879552  f70daefaf270b94174f01e5aab5b1d7d28cde06afc0dc3...   
1879553  f70daefaf270b94174f01e5aab5b1d7d28cde06afc0dc3...   
1879554  f70daefaf270b94174f01e5aab5b1d7d28cde06afc0dc3...   

                                              cand_item_id  \
0        087e7f3dbec9326532f9fc784b68de294cb2d905d33bdf...   
1        5d81d4230671ed22e40ab9e05bb63fef5ad6766454714a...   
2        eff6d2ef2c44dc7361d389d3a9ce243e6e3079675c0b27...   
3      

In [ ]:
# for test

test_text = pd.read_parquet('df_test.parquet')
test_image = pd.read_parquet('test_image_dataset.parquet')

test_image = test_image[test_image['is_valid'] == 1]
test_image = test_image.drop_duplicates(subset='id')

merge_df = test_text.merge(
    test_image.rename(columns={
        'id': 'base_title_image', 
        'embeddings': 'base_img_embeddings'
    }),
    on='base_title_image', 
    how='left'
)

merge_df = merge_df.merge(
    test_image.rename(columns={
        'id': 'cand_title_image', 
        'embeddings': 'cand_img_embeddings'
    }),
    on='cand_title_image', 
    how='left'
)

merge_df = merge_df.drop(['is_valid_x', 'is_valid_y'], axis=1)
merge_df.to_parquet('all_data_test.parquet', index=False)

In [5]:
print(merge_df)

                                             base_item_id  \
0       c66017e8712f80266bbed8b68285ee1ed8bead00ebb450...   
1       c9a60f42a86c7d27df6c73be26bfef58efe58a71431955...   
2       81912ec6d2b220e0ed65413588acea910fe9b71229d281...   
3       0ca0d9733cf8b7d9aa097270c3e06bcd69a9061573d254...   
4       8d0ec90fb0e1ff6b71c7c77ad113282b27859d200406a0...   
...                                                   ...   
499995  dcabaad4d0cf1f31a147a2b627cadf17f5819d0e252223...   
499996  21e2f2bcb3a2a595950e64d03c50be26b19131014f50c8...   
499997  66756bcc9e712d7571a5e2bb7506b6e7bd5d18fffac376...   
499998  c27444fa3f0ee247b84ecb9c58a3a691336697373572d5...   
499999  d35205358c9c901dc9864e17e6e69dfae8b8690818aa62...   

                                             cand_item_id  \
0       a4b4bc67f198e9ceda2e5d551f2d323cd4be6bc7fe3fb7...   
1       f6dee23c99b365055d2365ccb7b60d52369a11e753ebf6...   
2       fdd213efde102f6dd1dce6830b323e9b8aa1484607bd91...   
3       4f232dbcb2884c5